In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import geodesic
from sklearn.cluster import DBSCAN
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import time
from sklearn.preprocessing import StandardScaler

from utils.functions import clean_uber_data



NameError: name '_C' is not defined

In [ ]:
csv_path = "../data/uber.csv"
df = pd.read_csv(csv_path)

In [ ]:
# Define bounds for New York City
LAT_MAX = 40.9
LAT_MIN = 40.6
LONG_MAX = -73.6
LONG_MIN = -74.2
assert (LAT_MAX >= -90 and LAT_MAX <= 90) and (LAT_MIN >= -90 and LAT_MIN <= 90) and (LONG_MAX >= -90 and LONG_MAX <= 90) and (LONG_MIN >= -90 and LONG_MIN <= 90)
assert LAT_MIN < LAT_MAX and LONG_MIN < LONG_MAX

# Assuming df is your original DataFrame
df_cleaned, dropped_rows = clean_uber_data(df, LONG_MIN, LONG_MAX, LAT_MIN, LAT_MAX)

In [ ]:
# Feature scaling
scaler = StandardScaler()
X = scaler.fit_transform(df_cleaned[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'distance_miles', 'pickup_hour']])

In [ ]:
# Define the model class (same as the one used in training)
class FareEstimator(nn.Module):
    def __init__(self, input_size):
        super(FareEstimator, self).__init__()
        self.linear = nn.Linear(input_size, 1)
    
    def forward(self, x):
        return self.linear(x)

# Load the model
input_size = X.shape[1]  # Same input size used in training
model = FareEstimator(input_size)

# Load the saved weights
model_path = '../models/fare_estimator.pth'
model.load_state_dict(torch.load(model_path))
model.eval()  # Set to evaluation mode

print(f"Model loaded from {model_path}")

In [ ]:
# Example: Predict the fare for a new or existing data point

# Take one row from the test set or a new input (you can adjust this based on your input)
example_input = X[0]  # Take the first example row

# Convert to a tensor
example_input_tensor = torch.tensor(example_input, dtype=torch.float32).unsqueeze(0)  # Add batch dimension

# Make prediction
with torch.no_grad():
    predicted_fare = model(example_input_tensor)

print(f"Predicted fare: {predicted_fare.item()}")

In [3]:
# Generate predictions for the first N samples in the test set
N = 100  # Number of test samples to visualize
predicted_fares = []
actual_fares = []

# Iterate over the first N examples from the test set
with torch.no_grad():
    for i in range(N):
        # Get the actual input data
        example_input = X_test[i]  # Take the i-th example from the test set
        
        # Convert to tensor and add batch dimension
        example_input_tensor = torch.tensor(example_input, dtype=torch.float32).unsqueeze(0)
        
        # Make prediction
        predicted_fare = model(example_input_tensor)
        
        # Store the predicted fare and the actual fare
        predicted_fares.append(predicted_fare.item())
        actual_fares.append(y_test[i])

# Plotting the results
plt.figure(figsize=(10, 6))

# Plot actual vs predicted fares
plt.plot(range(N), actual_fares, label='Actual Fares', color='blue')
plt.plot(range(N), predicted_fares, label='Predicted Fares', color='orange')

# Add labels and title
plt.xlabel('Sample Index')
plt.ylabel('Fare Amount')
plt.title('Predicted vs Actual Fares for Test Data')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


NameError: name 'torch' is not defined